# Qiita

see: https://qiita.com/nkmk/items/fbac13cd05b80334eb2b

see: https://qiita.com/SugarShootingStar/items/bba0fab1e55936e7234a

データ取得

In [ ]:
import os
import time

import pandas as pd
import requests

url = 'https://qiita.com/api/v2/items'

h = {'Authorization': 'Bearer <token>'}

start = '2017-12-15'
end = '2018-12-15'

date_list = [d.strftime('%Y-%m-%d') for d in pd.date_range(start, end, freq='SM')]

start_list = date_list[:-1]
end_list = date_list[1:]

result_dir_path = 'results'
sleep_sec = 3.6


def get_simple_df(df):
    df['tags_str'] = df['tags'].apply(
        lambda tags: ','.join(tag['name'] for tag in tags if not tag['name'] == '\x00')
    )
    df['title'] = df['title'].str.replace('\r', '')
    return df[['title', 'id', 'created_at', 'updated_at','likes_count', 'comments_count', 'tags_str', 'user_id', 'user_permanent_id', 'url']]


for start, end in zip(start_list, end_list):
    p = {
        'per_page': 100,
        'query': 'created:>{} created:<{}'.format(start, end)
    }
    
    print("started")

    time.sleep(sleep_sec)
    r = requests.get(url, params=p, headers=h)
    total_count = int(r.headers['Total-Count'])

    print(total_count)
    
    if total_count == 0:
        continue

    df_list = [get_simple_df(pd.io.json.json_normalize(r.json(), sep='_'))]
    
    print("a")

    if total_count > 100:
        for i in range(2, (total_count - 1) // 100 + 2):
            print(i)
            p['page'] = i
            time.sleep(sleep_sec)
            r = requests.get(url, params=p, headers=h)
            df_list.append(get_simple_df(pd.io.json.json_normalize(r.json(), sep='_')))

    a = pd.concat(df_list, ignore_index=True)
    print(len(a))
    a.to_csv(os.path.join(result_dir_path, start + '.csv'), index=False)

df_all = [pd.read_csv(os.path.join(result_dir_path, start_date + '.csv')) for start_date in start_list]
pd.concat(df_all, ignore_index=True).to_csv(os.path.join(result_dir_path, 'summary.csv'), index=False)